# MNIST classification

오늘부터 pytorch로 딥러닝을 다시 시작해보려 한다.
요새 코딩을 통 하지 않아 까먹기도 했고, 처음부터 착착착 쌓아가고 싶은 마음도 있었다.
나만의 스크래치 코딩을 통해..

계획은 이렇다.

```
1. classification > MNIST > CNN, RNN, LSTM, transformer
2. classification > IMDB > LSTM, transformer
3. classification > CIFAR-10 > CNN, RNN, LSTM, transformer
4. generation > text data > seq2seq, transformer, BERT
5. 그 외.. (grad-CAM, GAN, time-series, anomaly-d, reinforcement ..)
```


이렇게 다양한 task에 대해 착착 쌓아가고 싶다.

바쁜 스케줄 속에서, 꾸준히 이걸 해나가려면, 아마 잠자는 시간을 쪼개야 할 것이다.
잠자는 건.. 주말에 하루 몰아서 푹 자자.

숲 속에 있다보니, 멍 때리면서 가끔 지금의 시간이 얼마나 소중한지를 잊곤 한다.
잊지 말자. 그 기억, 그 깨달음이 너의 힘인 데, 그걸 잊으면 넌 모두 잃어버리는 것이나 마찬가지다.

여튼, 오늘부터 MNIST 데이터 셋을 받아, digit image classificaiton task를 풀어볼 것이다.

**이번엔 지난번에 이어 CNN구조를 활용할 것이다.**


지난 FC 구조를 활용한 작업 후, 모듈화를 진행했다. 이번엔 이 모듈화된 .py들을 활용해 진행할 것이다. 코드는 github에서 확인 가능하다.

In [1]:
import os
os.getcwd()

'/home/jeongseobkim/바탕화면/pytorch_reboost/MNIST_classification'

In [2]:
!ls

 0.exercise.ipynb		       __pycache__      models
'1. MNIST_classification_FC.ipynb'     data	        train.py
'2. MNIST_classification_CNN.ipynb'    dataloader.py    trainer.py
'3. MNIST_classification_CNN.ipynb'    model_cnn.pth    utils.py
'3. MNIST_classification_LSTM.ipynb'   model_lstm.pth


In [3]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from dataloader import *
from models.model_fc import *
from models.model_cnn import *
from models.model_lstm import *

# train

In [4]:
# !python train.py --model lstm --model_fn model_lstm.pth

**Ipython에선 위 처럼 에러가 났다. 그래서 terminal에서 돌려서 model_lstm.pth를 얻기로 했다.**

# test

In [5]:
import argparse
config = argparse.Namespace()

config.model = 'cnn'
config.model_fn = 'model_cnn.pth'
config.gpu_id = 0 if torch.cuda.is_available() else -1
config.batch_size = 32
config.optim = 'adam'
config.lr = 1e-2
config.train_ratio = .8
config.n_epochs = 30
config.batch_norm = True
config.hidden_size = 64
config.num_layers = 3

config

Namespace(batch_norm=True, batch_size=32, gpu_id=0, hidden_size=64, lr=0.01, model='cnn', model_fn='model_cnn.pth', n_epochs=30, num_layers=3, optim='adam', train_ratio=0.8)

In [6]:
# test data load

_, _, test_loader = get_loaders(config)

len(test_loader)

313

In [7]:
for x, y in test_loader:
    print(f"x shape: {x.shape}, y shape : {y.shape}")
    break

x shape: torch.Size([32, 28, 28]), y shape : torch.Size([32])


In [8]:
# model load

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model = torch.load('model_cnn.pth', map_location=device)

# model['model']

In [9]:
# model['config']

In [10]:
# model['opt']

In [11]:
model_dict = model['model']
model_config = model['config']

# test

In [12]:
model = Cnn_classifier(10).to(device)

model.load_state_dict(model_dict)

correct_cnt, y_cnt = 0, 0

model.eval()
with torch.no_grad():
    for x, y in test_loader:
        x = x.type(torch.cuda.FloatTensor)     
        x, y = x.to(device), y.to(device)
        
        # |x| = (32,28,28)
        y_hat = model(x).to(device)
        # |y_hat| = (32,10)
        # |y| = (32)
        
        correct_cnt += float((y == torch.argmax(y_hat, dim=-1)).sum())
        y_cnt += len(y)

    accuracy = correct_cnt / y_cnt
    
    print("accuracy : %.3f" % accuracy)

accuracy : 0.994


test accuracy가 0.994로 매우 높게 나왔다.